In [1]:
import os
import torch
from torchsummary import summary
from transformers import AutoTokenizer, AutoModel

from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from datasets import load_dataset
from toxigen import label_annotations

In [2]:
cols = ['text', 'label']

In [3]:
implicit_df = pd.read_csv('../data/implicit-hate-corpus/implicit_hate_v1_stg3_posts.tsv', sep="\t")
implicit_df['label'] = 0
implicit_df = implicit_df.rename(columns={"post": "text"})
implicit_df_v2 = implicit_df[cols]
implicit_df_v2.head()

,text,label
0,"rt "" white privilege "" "" is when the gun jam...",0
1,if you are white in that state watch yourself !,0
2,so the 500 % increase in h1b visas ? yeah i'm ...,0
3,rt antiwhites expose their hatred of white ki...,0
4,capitalism like communism is an enemy of the...,0


In [4]:
toxigen_data = load_dataset("toxigen/toxigen-data", name="train")
toxigen_df = pd.DataFrame.from_dict(toxigen_data['train'])
alice_df = toxigen_df[toxigen_df['generation_method'] == 'ALICE']

alice_df['label'] = 1
alice_df_v2 = alice_df.rename(columns={"generation": "text"})
alice_df_v2 = alice_df_v2[cols]
alice_df_v2.head()

C:\Users\steve\AppData\Local\Temp\ipykernel_56264\526951477.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alice_df['label'] = 1


,text,label
0,"if youre white, you should be proud to be whit...",1
1,the best thing that could have happened to the...,1
2,theres no hope for the future because theres n...,1
3,"if youre white, you should be proud of it.",1
4,"no matter how much money or power you have, if...",1


In [11]:
print(implicit_df_v2.shape, alice_df_v2.shape)
ai_df = pd.concat([implicit_df_v2, alice_df_v2])
subai_df = ai_df.sample(frac=.01)
ai_df.shape, subai_df.shape

(6359, 2) (9809, 2)


((16168, 2), (162, 2))

In [13]:
subai_df['label'].value_counts()

label
1    104
0     58
Name: count, dtype: int64

In [ ]:
for layer in model.children():
    print(layer)

In [6]:
# Define the custom HateBERT model
class CustomHateBERTModel(nn.Module):
    def __init__(self, pretrained_model_name, num_classes, dropout_rate=0.3):
        super(CustomHateBERTModel, self).__init__()
        # Load the pre-trained HateBERT model
        self.encoder = AutoModel.from_pretrained(pretrained_model_name)
        hidden_size = self.encoder.config.hidden_size
        
        # Add custom layers for classification
        self.pooling = nn.AdaptiveAvgPool1d(1)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        # Forward pass through HateBERT
        encoder_outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = encoder_outputs.last_hidden_state
        pooled_output = self.pooling(last_hidden_state.permute(0, 2, 1)).squeeze(-1)

        # Apply dropout and dense layer
        dropped_output = self.dropout(pooled_output)
        logits = self.fc(dropped_output)
        return logits

# # Initialize the model
# pretrained_model_name = "GroNLP/hateBERT"
# num_classes = 3  # Replace with your number of classes
# model = CustomHateBERTModel(pretrained_model_name, num_classes)

# # Loss function: Categorical Cross-Entropy
# criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification

# # Optimizer: Adam with learning rate 0.001
# optimizer = Adam(model.parameters(), lr=0.001)

# Metric: Accuracy function
def accuracy(preds, labels):
    _, predictions = torch.max(preds, dim=1)
    return (predictions == labels).sum().item() / labels.size(0)

# Example Dataset (for demonstration purposes)
class ExampleDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, truncation=True, padding="max_length", max_length=self.max_len, return_tensors="pt")
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long),
        }

# Training loop
def train_model(model, data_loader, optimizer, criterion, device):
    model = model.to(device)
    model.train()

    total_loss = 0
    total_acc = 0

    for batch in data_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask)

        # Compute loss
        loss = criterion(outputs, labels)
        total_loss += loss.item()

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Compute accuracy
        acc = accuracy(outputs, labels)
        total_acc += acc

    return total_loss / len(data_loader), total_acc / len(data_loader)

# Example usage
# tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
# texts = ["This is a sample text.", "Another example of text."]
# labels = [0, 1]  # Example labels (multi-class indices)

# dataset = ExampleDataset(texts, labels, tokenizer, max_len=128)
# data_loader = DataLoader(dataset, batch_size=2, shuffle=True)

# # Training configuration
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# epochs = 3

# for epoch in range(epochs):
#     train_loss, train_acc = train_model(model, data_loader, optimizer, criterion, device)
#     print(f"Epoch {epoch + 1}/{epochs}")
#     print(f"Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")


In [7]:
class TextClassificationDataset(Dataset):
    """PyTorch Dataset for text classification using a Hugging Face tokenizer.

    Attributes:
        texts (list): A list of input text strings.
        labels (list): A list of integer labels corresponding to the input texts.
        tokenizer (AutoTokenizer): Tokenizer from the Hugging Face Transformers library.
        max_len (int): Maximum length of tokenized input sequences.
    """
    def __init__(self, texts, labels, tokenizer, max_len):
        """Initializes the dataset.

        Args:
            texts (list): A list of strings containing the input texts.
            labels (list): A list of strings or integers containing the labels.
            tokenizer (AutoTokenizer): Tokenizer from the Hugging Face Transformers library.
            max_len (int): Maximum length of tokenized input sequences.
        """
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        """Returns the number of samples in the dataset."""
        return len(self.texts)

    def __getitem__(self, idx):
        """Fetches a single data point at the given index.

        Args:
            idx (int): Index of the data point to fetch.

        Returns:
            dict: A dictionary containing input IDs, attention masks, and the label.
        """
        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenize the text
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long)
        }

def create_text_classification_dataset(df, 
                                       #label_mapping,
                                       tokenizer_name, max_len):
    """Creates a PyTorch dataset from a pandas dataframe for text classification.

    Args:
        df (pd.DataFrame): DataFrame containing two columns: the first column has the text, and the second column has the labels.
        label_mapping (dict): A dictionary mapping string labels to integer class indices.
        tokenizer_name (str): Name of the pre-trained tokenizer (e.g., "GroNLP/hateBERT").
        max_len (int): Maximum length of tokenized input sequences.

    Returns:
        TextClassificationDataset: A PyTorch dataset ready for training.
    """

    # Extract texts and map string labels to integers
    texts = df.iloc[:, 0].tolist()
    # labels = df.iloc[:, 1].map(label_mapping).tolist()
    labels = df.iloc[:, 1].tolist()

    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    # Create and return the dataset
    return TextClassificationDataset(texts, labels, tokenizer, max_len)


In [8]:
# Initialize the model
pretrained_model_name = "GroNLP/hateBERT"
num_classes = 2  # Replace with your number of classes
model = CustomHateBERTModel(pretrained_model_name, num_classes)

# Loss function: Categorical Cross-Entropy
criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification

# Optimizer: Adam with learning rate 0.001
optimizer = Adam(model.parameters(), lr=0.001)

C:\Users\steve\.conda\envs\ai-hate\Lib\site-packages\transformers\modeling_utils.py:1435: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_arch

In [14]:
text_ds = create_text_classification_dataset(subai_df, 
                                             #label_mapper, 
                                             tokenizer_name="GroNLP/hateBERT", max_len=128)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
#dataset = ExampleDataset(texts, labels, tokenizer, max_len=128)
data_loader = DataLoader(text_ds, batch_size=2, shuffle=True)

# Training configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 3

for epoch in range(epochs):
    train_loss, train_acc = train_model(model, data_loader, optimizer, criterion, device)
    print(f"Epoch {epoch + 1}/{epochs}")
    print(f"Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")

Epoch 1/3
Loss: 0.6989, Accuracy: 0.5494
Epoch 2/3
Loss: 0.6703, Accuracy: 0.6420
Epoch 3/3
Loss: 0.6799, Accuracy: 0.6296
